In [1]:
# Find the latest version of spark from https://downloads.apache.org/spark/
# https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz


# Install Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download and install Apache Spark
# Ensure this URL points to a valid Spark tarball
spark_version = '3.5.0'
hadoop_version = '3'
spark_filename = f'spark-{spark_version}-bin-hadoop{hadoop_version}'
spark_url = f'https://downloads.apache.org/spark/spark-{spark_version}/{spark_filename}.tgz'

!wget -q {spark_url}
!tar xf {spark_filename}.tgz

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_filename}"

# Install findspark
!pip install -q findspark

# Import findspark and initialize.
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [633 kB]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,035 kB]
Get:13 http://security.ubuntu.com/u

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles

url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

spark.sparkContext.addFile(url)

df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)

df.show()

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [4]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView("home_sales")

start_time = time.time()

query1 = """

select
      YEAR(date), ROUND(avg(price))
from
      home_sales
where
      bedrooms = '4'
group by
      YEAR(date)

"""

query2 = """

select
      date_built, ROUND(avg(price))
from
      home_sales
where
      bedrooms = '3' and bathrooms = '3'
group by
      date_built

"""

query3 = """

select
      date_built, ROUND(avg(price))
from
      home_sales
where
      bedrooms = '3' and bathrooms = '3' and floors = '2' and sqft_living <= '2000'
group by
      date_built

"""

query4 = """

select
      view, ROUND(avg(price))
from
      home_sales
where
      price <= '350000'
group by
      view

"""


#spark.sql(query1).show()
#spark.sql(query2).show()
#spark.sql(query3).show()
spark.sql(query4).show()

print(f"time passed is :{time.time() - start_time}")

+----+--------------------+
|view|round(avg(price), 0)|
+----+--------------------+
|   7|            239961.0|
|  15|            236561.0|
|  11|            231355.0|
|  29|            235915.0|
|  42|            235743.0|
|  87|           1391853.0|
|   3|            234471.0|
|  30|            234214.0|
|  34|            232112.0|
|   8|            234155.0|
|  22|            233356.0|
|  28|            233239.0|
|  85|           1386319.0|
|  35|            231676.0|
|  16|            235060.0|
|   0|            238377.0|
|  98|           1363392.0|
|  47|            240530.0|
|  99|           1358635.0|
|  96|           1403339.0|
+----+--------------------+
only showing top 20 rows

time passed is :3.9358839988708496


In [5]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
spark.sql(query1).show()

+----------+--------------------+
|year(date)|round(avg(price), 0)|
+----------+--------------------+
|      2022|            296364.0|
|      2019|            300264.0|
|      2020|            298354.0|
|      2021|            301819.0|
+----------+--------------------+



In [6]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql(query2).show()


+----------+--------------------+
|date_built|round(avg(price), 0)|
+----------+--------------------+
|      2016|            290555.0|
|      2012|            293683.0|
|      2017|            292677.0|
|      2014|            290852.0|
|      2013|            295962.0|
|      2011|            291117.0|
|      2015|            288770.0|
|      2010|            292860.0|
+----------+--------------------+



In [7]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
spark.sql(query3).show()


+----------+--------------------+
|date_built|round(avg(price), 0)|
+----------+--------------------+
|      2016|            281115.0|
|      2012|            285400.0|
|      2017|            283691.0|
|      2014|            290731.0|
|      2013|            286275.0|
|      2011|            285979.0|
|      2015|            285707.0|
|      2010|            275609.0|
+----------+--------------------+



In [8]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql(query4).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 0)|
+----+--------------------+
|   7|            239961.0|
|  15|            236561.0|
|  11|            231355.0|
|  29|            235915.0|
|  42|            235743.0|
|  87|           1391853.0|
|   3|            234471.0|
|  30|            234214.0|
|  34|            232112.0|
|   8|            234155.0|
|  22|            233356.0|
|  28|            233239.0|
|  85|           1386319.0|
|  35|            231676.0|
|  16|            235060.0|
|   0|            238377.0|
|  98|           1363392.0|
|  47|            240530.0|
|  99|           1358635.0|
|  96|           1403339.0|
+----+--------------------+
only showing top 20 rows

--- 0.6458089351654053 seconds ---


In [9]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [10]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [11]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()
query5 = """

select
      view, ROUND(avg(price))
from
      home_sales
where
      price <= '350000'
group by
      view

"""

spark.sql(query5).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 0)|
+----+--------------------+
|   7|            239961.0|
|  15|            236561.0|
|  11|            231355.0|
|  29|            235915.0|
|  42|            235743.0|
|  87|           1391853.0|
|   3|            234471.0|
|  30|            234214.0|
|  34|            232112.0|
|   8|            234155.0|
|  22|            233356.0|
|  28|            233239.0|
|  85|           1386319.0|
|  35|            231676.0|
|  16|            235060.0|
|   0|            238377.0|
|  98|           1363392.0|
|  47|            240530.0|
|  99|           1358635.0|
|  96|           1403339.0|
+----+--------------------+
only showing top 20 rows

--- 0.5978062152862549 seconds ---


In [12]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
spark.conf.set("spark.sql.shuffle.partitions", 8)
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_parquet")

In [13]:
# 11. Read the formatted parquet data.
df_parquet = spark.read.parquet("home_sales_parquet")

In [14]:
# 12. Create a temporary table for the parquet data.
df_parquet.createOrReplaceTempView("home_sales_parquet_view")

In [15]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

query6 = """

select
      view, ROUND(avg(price))
from
      home_sales_parquet_view
where
      price <= '350000'
group by
      view

"""
spark.sql(query6).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 0)|
+----+--------------------+
|   1|            242194.0|
|  44|            233405.0|
|  43|            237204.0|
|  80|           1378807.0|
|  85|           1386319.0|
|  87|           1391853.0|
|   5|            230727.0|
|  21|            233605.0|
|  14|            236689.0|
|  35|            231676.0|
|  16|            235060.0|
|  48|            234616.0|
|  89|           1370636.0|
|  93|           1397043.0|
|  92|           1357087.0|
|  77|           1355392.0|
|  95|           1353350.0|
|   3|            234471.0|
|   8|            234155.0|
|  36|            227436.0|
+----+--------------------+
only showing top 20 rows

--- 0.8338413238525391 seconds ---


In [16]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [17]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached("home_sales")


False